In [16]:
# optimizer with scaling and lin reg

import pandas as pd
import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
from sklearn.neural_network import MLPRegressor

df = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')

df = df[:1350]

dfTerran = df[df["fraction"] == "T"]

X = dfTerran[["total_workers","total_army"]]
y = dfTerran["resource_mining"]

# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

feat_transform = make_column_transformer(
    (StandardScaler(), ["total_workers", "total_army", "time"]),
)

scaler = StandardScaler()
lin_reg = make_pipeline(scaler, MLPRegressor([20]*2, max_iter=10000, random_state=1))
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)
print(f"The R^2 value in the test set is {r2_score(y_test, y_pred)}")

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]

dfTerranOpt = data[data["fraction"] == "T"]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfTerranOpt = dfTerranOpt.drop(columns=[
                               "total_workers", "player", "replay_filename", "winner", "fraction", "Unnamed: 0", "total_army_value", "resource_consumption", "time"])


# create Model
m = gp.Model("Worker optimizer")

gppd.set_interactive()

# create variables
av = gppd.add_vars(m, lb=-gp.GRB.INFINITY,
                   pandas_obj=dfTerranOpt, name="total_army_value")
w = gppd.add_vars(m, pandas_obj=dfTerranOpt,
                  name="total_workers", vtype=gp.GRB.INTEGER)
r_ges = gppd.add_vars(m, pandas_obj=dfTerranOpt,
                      name="resource_mining")
a = gppd.add_vars(m, pandas_obj=dfTerranOpt,
                  name="total_army", vtype=gp.GRB.INTEGER)
r_a = gppd.add_vars(m, pandas_obj=dfTerranOpt,
                    name="resource_per_unit")

m.update()
dfTerranOpt = dfTerranOpt.drop(columns=["resource_mining"])

feats = dfTerranOpt
feats = pd.concat([w, feats], axis=1)
print(feats.head)

m.setObjective(av.sum(), gp.GRB.MAXIMIZE)
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0)
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, 200)
gppd.add_constrs(m, a * r_a, gp.GRB.EQUAL, av)
gppd.add_constrs(m, r_a, gp.GRB.GREATER_EQUAL, 50)
gppd.add_constrs(m, r_a, gp.GRB.LESS_EQUAL, 100)
gppd.add_constrs(m, r_ges - 50, gp.GRB.EQUAL, w)
gppd.add_constrs(m, av + 50 * w, gp.GRB.LESS_EQUAL, r_ges)
m.update()
pred_constr = add_predictor_constr(m, lin_reg, feats, r_ges)
pred_constr.print_stats()
m.optimize()
outputs = pd.concat([pred_constr.input_values, av], axis=1)
print(outputs)
"""s#



feats = feats.drop(columns=["time"])
print(feats)

# pred_constr = add_predictor_constr(m, lin_reg, feats, av)
# m.Params.NonConvex = 2


print(w.gppd.X)

print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)







dfTerranOpt = dfTerranOpt.loc[:, features]

m.update()

dfTerranOpt[:10]





"""


The R^2 value in the test set is 0.8657547780753174
<bound method NDFrame.head of                          total_workers  total_army
1356  <gurobi.Var total_workers[1356]>          76
1358  <gurobi.Var total_workers[1358]>          39
1364  <gurobi.Var total_workers[1364]>          20
1366  <gurobi.Var total_workers[1366]>          30
1368  <gurobi.Var total_workers[1368]>           9
...                                ...         ...
2648  <gurobi.Var total_workers[2648]>          45
2649  <gurobi.Var total_workers[2649]>          40
2650  <gurobi.Var total_workers[2650]>          27
2651  <gurobi.Var total_workers[2651]>           4
2652  <gurobi.Var total_workers[2652]>          19

[476 rows x 2 columns]>
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Model for pipe1:
39508 variables
20468 constraints
19040 general constraints
Input has shape (476, 2)
Output has shape (476, 1)

Pipeline has 2 steps:

---------------------------------------------------

GurobiError: Unable to retrieve attribute 'X'